In [1]:
!pip install torch torchvision kaggle tqdm


In [2]:
import kagglehub

path = kagglehub.dataset_download(
    "anthonytherrien/image-classification-64-classes-animal"
)

print("Path to dataset files:", path)


100%|██████████| 5.71G/5.71G [00:52<00:00, 118MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/anthonytherrien/image-classification-64-classes-animal/versions/1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import os

image_dir = os.path.join(path, "image")
print(os.listdir(image_dir)[:20])
items = os.listdir(image_dir)
folders = [i for i in items if os.path.isdir(os.path.join(image_dir, i))]
files = [i for i in items if os.path.isfile(os.path.join(image_dir, i))]

print("Folders:", folders[:10])
print("Files:", files[:10])


['cat', 'walrus', 'leopard', 'hawk', 'wolf', 'peacock', 'blackbird', 'cheetah', 'goose', 'sheep', 'lynx', 'hippopotamus', 'porcupine', 'mole', 'lizard', 'bison', 'lemur', 'eagle', 'hyena', 'snail']
Folders: ['cat', 'walrus', 'leopard', 'hawk', 'wolf', 'peacock', 'blackbird', 'cheetah', 'goose', 'sheep']
Files: []


In [6]:
BASE_DIR = os.path.join(path, "image")


In [8]:
TARGET_CLASSES = ["elephant", "deer", "cow"]


In [9]:
import os, shutil, random

OUTPUT_DIR = "dataset"

for split in ["train", "val"]:
    for cls in TARGET_CLASSES:
        os.makedirs(f"{OUTPUT_DIR}/{split}/{cls}", exist_ok=True)


In [11]:
for cls in TARGET_CLASSES:
    img_dir = os.path.join(BASE_DIR, cls)
    images = os.listdir(img_dir)
    random.shuffle(images)

    split_idx = int(0.8 * len(images))
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    for img in train_imgs:
        shutil.copy(
            os.path.join(img_dir, img),
            f"{OUTPUT_DIR}/train/{cls}/{img}"
        )

    for img in val_imgs:
        shutil.copy(
            os.path.join(img_dir, img),
            f"{OUTPUT_DIR}/val/{cls}/{img}"
        )

print("✅ Dataset prepared successfully")


✅ Dataset prepared successfully


In [17]:
import shutil

# Syntax: shutil.make_archive(output_filename, format, source_directory)
shutil.make_archive('dataset_download', 'zip', '/content/dataset')

'/content/dataset_download.zip'

In [19]:
from google.colab import drive
drive.mount('/content/drive')
!cp dataset_download.zip /content/drive/MyDrive/dataset_download.zip


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
for split in ["train", "val"]:
    for cls in TARGET_CLASSES:
        count = len(os.listdir(f"{OUTPUT_DIR}/{split}/{cls}"))
        print(split, cls, count)


train elephant 230
train deer 172
train cow 172
val elephant 58
val deer 44
val cow 44


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
EPOCHS = 12
NUM_CLASSES = 3


In [22]:
train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor()
])

val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])


In [23]:
train_ds = datasets.ImageFolder("dataset/train", transform=train_tf)
val_ds = datasets.ImageFolder("dataset/val", transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

print("Class mapping:", train_ds.class_to_idx)


Class mapping: {'cow': 0, 'deer': 1, 'elephant': 2}


In [24]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, NUM_CLASSES)
model = model.to(DEVICE)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 116MB/s] 


In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_loader):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")


100%|██████████| 18/18 [00:14<00:00,  1.24it/s]


Epoch 1 Loss: 0.1156


100%|██████████| 18/18 [00:10<00:00,  1.64it/s]


Epoch 2 Loss: 0.0014


100%|██████████| 18/18 [00:11<00:00,  1.62it/s]


...

Epoch 10 Loss: 0.0003


100%|██████████| 18/18 [00:10<00:00,  1.67it/s]


Epoch 11 Loss: 0.0001


100%|██████████| 18/18 [00:10<00:00,  1.69it/s]

Epoch 12 Loss: 0.0002


In [26]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100*correct/total:.2f}%")


Validation Accuracy: 100.00%


In [29]:
torch.save(model.state_dict(), "species_classifier.pt")


In [31]:
from google.colab import files
files.download("species_classifier.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>